In [223]:
# Подключаем библиотеки

# Для парсинга файла html
from bs4 import BeautifulSoup 
import re
import os
import fnmatch
import numpy as np

# Создаем список, элементы которого будут представлять текст статьи на опредленную тему. 
collection = []

# Всего областей 5: a) биология, b) информационные технологии, c) история, d) менеджмент, 
# e) архитектура и строительство
# Создадим список, в котором каждому номеру документа соответствует область
field = []


# Читаем все файлы из папки dataset
for filename in os.listdir("dataset"):
    with open(os.path.join("dataset", filename), "rb") as f:
        contents = f.read().decode(errors='replace')
        soup = BeautifulSoup(contents, 'html.parser')
        
        a = soup.find_all('p', style = re.compile("[\D+]?text-align: justify;"))
        s = ''
        
        for i in range(len(a)):
        
# Некоторые атрибуты в файле имели пустые значения и при записи почему-то записывались как \xa0, поэтому
# пришлось проводить проверку на то, чтобы a[i].text != '\xa0' и только потом записывать в общую строку
            if a[i].text != '\xa0':
                s += ' ' + re.sub('[a-zA-Z0-9%-.,:—;"!№/\[\]\\\'?»«–’”“℃]','',a[i].text.lower())
         
    
# При парсинге не получалось удалить некоторые объекты, пришлось ручками.
    s = s.replace('\u200b','')
    s = s.replace('\xad', '')
    s = s.replace('\xa0', '')
    collection.append(s)
    
    if fnmatch.fnmatch(filename, 'a*.html'):
        field.append('Биология')
    elif fnmatch.fnmatch(filename, 'b*.html'):
        field.append('Информационные технологии')
    elif fnmatch.fnmatch(filename, 'c*.html'):
        field.append('История')
    elif fnmatch.fnmatch(filename, 'd*.html'):
        field.append('Менеджмент')
    elif fnmatch.fnmatch(filename, 'e*.html'):
        field.append('Архитектура')
    else: field.append('1')
    
collection.remove(collection[14])
field.remove(field[14])

# field

# Мощность датасета и пример одного элемента 
# print(len(collection))
# print(collection[108])

# Предобработка данных

Данные уже :
1. приведены к нижнему регистру
2. отчищены от пунктуационных знаков
3. отчищены от спец.знаков

В этом разделе:
1. избавимся от стоп-слов
2. проведем лемматизацию

In [224]:
import math

def idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [225]:
collection_split = [collection[i].split() for i in range(len(collection))]

data = []
for i in range(len(collection_split)):
    data.append(list(filter(lambda x: idf(x,collection_split) > 0.5, collection_split[i])))
# data[0]

In [226]:
# Напишем функцию для лемматизации текста и выведем коллекцию статей снова

import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def lemmat(data):
    res_data = []
    for words in data:
        res_line = []
        for word in words:
            res_line.append(morph.parse(word)[0].normal_form)
        res_data.append(' '.join(res_line)) 
    return res_data

collection = lemmat(data)

In [227]:
data = [collection[i].split() for i in range(len(collection))]
# data

set_data = set(data[0])
for i in range(len(data)):
    set_data |= set(data[i])
               
len(set_data)
# set_data

8898

# TF-IDF

In [228]:
import collections

def tf(word, text):
    w_count = 0
    for i in text:
        if i == word:
            w_count += 1
    return w_count / float(len(text))

In [229]:
# На вход подается слово, документ и вся коллекция (оба в виде списков слов)
# На выходе - tf-idf значение для слова в документе

def tf_idf(word, text, collection):
    return tf(word, text) * idf(word, collection)

# Построение обратного индекса

In [230]:
inverted = {}

for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j] not in inverted:
            inverted[data[i][j]] = [i]
        elif i not in inverted[data[i][j]]:
            inverted[data[i][j]].append(i)
            
# inverted

In [231]:
def fast_search(text, set_data):
    set_w = set()
    for word in text:
        if word in inverted:
            set_w |= set(inverted[word])  # Ищем объединение всех документов
    return set_w

# Поиск по запросу

In [250]:
# Введем запрос и преобразуем его
search = 'Виды растений и животных'
search = lemmat([search.lower().split()])[0].split()
search

['вид', 'растение', 'и', 'животное']

In [251]:
# Список всех документов, в которых используется хотя бы одно слово из запроса

search_docs = list(fast_search(search, set_data))
# search_docs

# Ранжирование (tf-idf)

In [252]:
# Ранжируем, используя значения tf-idf

rang = {}
for i in search_docs:
    rang[i] = 0.0
    for word in search:
        if word in inverted:
            rang[i] += tf_idf(word, data[i], data)

# rang

In [253]:
res_rang = list(rang.items())
res_rang.sort(key = lambda i: i[1])

for index in reversed(res_rang):
    print(index[0], ':', index[1])

25 : 0.046812343130920725
82 : 0.036920854441251094
14 : 0.03544160508397547
73 : 0.027816851589629727
89 : 0.020028838599338638
105 : 0.019753478940449837
103 : 0.017446778904692387
50 : 0.016914366441242987
41 : 0.01623392483167811
49 : 0.013214390181444375
72 : 0.01280450626873142
96 : 0.00996067491212343
56 : 0.0038149626817522778
7 : 0.0032974898574089535
40 : 0.0031541007294546754
28 : 0.0023439138533112385
84 : 0.002293479233967332
43 : 0.002198326571605969
31 : 0.0015887121776961203
75 : 0.001415465142077286
104 : 0.0013960308380670716
77 : 0.0013320206904701096
52 : 0.0013320206904701096
32 : 0.0013054182801040875
57 : 0.0012250310767777314
39 : 0.0011709510100485802
80 : 0.001048222467758265
55 : 0.001048222467758265
94 : 0.0009935919808392675
68 : 0.0009935919808392675
70 : 0.0009903660328495296
0 : 0.0009850357528019005
23 : 0.0009652934750558706
10 : 0.0009652934750558706
4 : 0.0009652934750558706
62 : 0.0008841528641091454
13 : 0.000881597509010564
46 : 0.0008715221089075

In [254]:
for index in reversed(res_rang[-20:]):
    if index[1] != 0.0:
        print(field[index[0]])
        print(index[0])
#         print(collection[index[0]])
        print('\n')

Биология
25


Биология
82


Биология
14


Биология
73


Биология
89


Биология
105


Биология
103


Биология
50


Биология
41


Биология
49


Биология
72


Биология
96


Архитектура
56


Биология
7


Архитектура
40


Архитектура
28


Информационные технологии
84


История
43


Менеджмент
31


Архитектура
75




# Ранжирование (Word2Vec)

In [237]:
# Напишем функцию для векторизации документ. Усредним оценки для всех слов в предложении

def vec_w2v(model, text):
    sum_ = 0
    for j in range(len(text)):
        sum_ += model.wv[text[j]]
    return sum_ / len(text)

In [238]:
# Введем запрос и преобразуем его
search_w2v = 'Виды растений и животных'
search_w2v = lemmat([search_w2v.lower().split()])[0].split()
search_w2v

['вид', 'растение', 'и', 'животное']

In [239]:
from gensim.models import Word2Vec

data.append(search_w2v)

word2vec = Word2Vec(data, min_count = 1)
vec_search_w2v = vec_w2v(word2vec, search_w2v)

In [240]:
# vec_w2v(word2vec, data[1])

In [241]:
# Напишем функцию, высчитывающую косинусное расстояние

def cos(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / np.linalg.norm(vec1) / np.linalg.norm(vec2)

In [242]:
# Построим список косинусного расстояния между вектором запроса и векторами статей

scalar = {}
for i in search_docs:
    scalar[i] = cos(vec_w2v(word2vec, data[i]), vec_search_w2v)

# scalar

In [243]:
# Создание словаря с номерами статей и соответствующими косинусными расстояниями. 
# Словарь сортируется по возрастанию косинусного расстояния. А выводит в обратном порядке, для наглядности.

list_rang_index = list(scalar.items())
list_rang_index.sort(key = lambda i: i[1])

for index in reversed(list_rang_index):
    print(index[0], ':', index[1])

82 : 0.999836
73 : 0.9998247
25 : 0.9998246
49 : 0.9998184
50 : 0.99981517
62 : 0.99981356
72 : 0.9998104
103 : 0.9998098
89 : 0.9998084
105 : 0.9998065
7 : 0.99980474
14 : 0.99980325
41 : 0.9998019
60 : 0.9997992
100 : 0.99979836
98 : 0.9997962
61 : 0.99979466
83 : 0.9997942
48 : 0.99979323
43 : 0.99979234
13 : 0.99979216
75 : 0.9997915
104 : 0.9997912
70 : 0.999791
40 : 0.9997905
58 : 0.9997894
88 : 0.99978834
45 : 0.99978834
80 : 0.9997858
55 : 0.9997858
28 : 0.9997853
57 : 0.9997846
99 : 0.99978393
46 : 0.99978304
24 : 0.9997829
31 : 0.99978244
0 : 0.9997797
96 : 0.99977905
10 : 0.99977905
77 : 0.9997784
52 : 0.9997784
32 : 0.99977726
79 : 0.99977565
38 : 0.999774
20 : 0.99977314
94 : 0.9997726
68 : 0.9997726
2 : 0.9997704
35 : 0.9997697
84 : 0.9997675
56 : 0.99976176
93 : 0.99975616
26 : 0.9997519
23 : 0.9997478
4 : 0.9997478
39 : 0.9997291


In [246]:
# Выведем первые 10 подходящих статей. Я вывожу области, из которых взяты статьи, чтобы предварительно показать
# качество работы модели. Вывод самих статей закомментирован. Чтобы посмотреть - раскомментируйте 7ю строку.
for index in reversed(list_rang_index[-10:-5]):
        print(field[index[0]])
        print(index[0])
        print(collection[index[0]])
        print('\n')

Архитектура
62
представлять себя крепость который земля почти остаться след крепость который сохраниться земля статья посвятить исследование крепость ям работа рассматриваться история градостроительный период археология крепость археология история фортификация один уникальный объект культурный наследие г кингисепп крепость ям исторический центр исторический код пространственный история крепость начинаться древний время главный источник изучение первый страница история ямгород летопись новгородский летописец сообщить закладка город камена р луга г яма дорожный станция возле речной переправа крепость ям являться оборонительный щит северозападный граница русь наряду другой крепость новгородский земля долгий крепость принимать себя удар ливонский орден поскольку она стать главный цитадель западный граница новгородский республика г крепость ям быть окончательно закрепить российский государство указ пётр подтверждать название город ямгород касаться строительство крепость здесь отметить тот ф

In [245]:
data.pop(len(data)-1)

'Сам запрос'

# Вывод

Видно, что разные методы выдают разные результаты. Word2Vec уже 6-ым выдает статью из области "Информационные технологии", в которой нет ни слова "растение", ни "животное". Есть только слово "вид", а оно не самое значимое в запросе. 

Ранжирование по TF-IDF работает лучше: первые 12 статей из области "Биология", а следующие содержат главные слова - "растение" или "животное"

In [263]:
# Введем запрос и преобразуем его
search = 'Вид тип разновидность семейство растений трава цветы и животных скот зверь'
search = lemmat([search.lower().split()])[0].split()
search

['вид',
 'тип',
 'разновидность',
 'семейство',
 'растение',
 'трава',
 'цветок',
 'и',
 'животное',
 'скот',
 'зверь']

In [256]:
# Ранжируем, используя значения tf-idf

rang = {}
for i in search_docs:
    rang[i] = 0.0
    for word in search:
        if word in inverted:
            rang[i] += tf_idf(word, data[i], data)

# rang

In [265]:
res_rang = list(rang.items())
res_rang.sort(key = lambda i: i[1])

# for index in reversed(res_rang):
#     print(index[0], ':', index[1])

In [262]:
for index in reversed(res_rang[-20:]):
    if index[1] != 0.0:
        print(field[index[0]])
        print(index[0])
#         print(collection[index[0]])
        print('\n')

Биология
50


Биология
49


Биология
25


Биология
105


Биология
82


Биология
14


Биология
73


Биология
103


Биология
89


Биология
41


Архитектура
26


Биология
72


Менеджмент
99


Биология
96


История
43


Информационные технологии
0


Информационные технологии
23


Информационные технологии
4


Архитектура
13


Архитектура
56


